## Structured Output

In [1]:
from crewai import Agent

agent = Agent(
    role="Senior Linguist",
    goal="Analyse the query and extract entity-relation-entity triplets",
    backstory="You are a senior linguist that is known for your analytical skills.",
    verbose=True
)

In [2]:
from crewai import Task

task = Task(
    description="""Analyse the query and return structured JSON
                   output in the form of
                   - entity
                   - relation
                   - entity

                   The query is: {query}
                   """,
    expected_output="A structured JSON object with the entity-relation-entity triplets",
    verbose=True,
    agent=agent
)

In [3]:
from crewai import Crew, Process

crew = Crew(
    agents=[agent],
    tasks=[task],
    process=Process.sequential,
    verbose=True
)

Exception while exporting Span batch.
Traceback (most recent call last):
  File "/Users/manideepbangaru/Documents/learnings/Agents_deepdive/agentEnv/lib/python3.11/site-packages/urllib3/connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/Users/manideepbangaru/Documents/learnings/Agents_deepdive/agentEnv/lib/python3.11/site-packages/urllib3/connection.py", line 516, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/http/client.py", line 1395, in getresponse
    response.begin()
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/http/client.py", line 325, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/http/client.py", line 286, in _re

In [4]:
response = crew.kickoff(inputs={
                          "query":"""Paris is the capital of France.
                                     The Eiffel Tower is in Paris."""
                               }
                       )

# Agent: Senior Linguist
## Task: Analyse the query and return structured JSON
                   output in the form of
                   - entity
                   - relation
                   - entity

                   The query is: Paris is the capital of France.
                                     The Eiffel Tower is in Paris.
                   


# Agent: Senior Linguist
## Final Answer: 
```json
[
    {
        "entity": "Paris",
        "relation": "is the capital of",
        "entity": "France"
    },
    {
        "entity": "The Eiffel Tower",
        "relation": "is in",
        "entity": "Paris"
    }
]
```




In [5]:
query = "Paris is the capital of France."

response = crew.kickoff(inputs={"query":query})

# Agent: Senior Linguist
## Task: Analyse the query and return structured JSON
                   output in the form of
                   - entity
                   - relation
                   - entity

                   The query is: Paris is the capital of France.
                   


# Agent: Senior Linguist
## Final Answer: 
```json
{
  "subject": "Paris",
  "relation": "is the capital of",
  "object": "France"
}
```




## Using PyDantic

In [6]:
from pydantic import BaseModel, Field

class EntityRelationEntity(BaseModel):
    entity: str = Field(description="The first entity in the triplet")
    relation: str = Field(description="The relation between the first and second entity")
    entity: str = Field(description="The second entity in the triplet")

In [7]:
from crewai import Agent

agent = Agent(
    role="Senior Linguist",
    goal="Analyse the query and extract entity-relation-entity triplets",
    backstory="You are a senior linguist that is known for your analytical skills.",
    verbose=True
)

In [8]:
from crewai import Task

task = Task(
    description="""Analyse the query and return structured JSON
                   output in the form of
                   - entity
                   - relation
                   - entity

                   The query is: {query}
                   """,
    expected_output="""A structured JSON object with the
                       entity-relation-entity triplets""",
    output_pydantic=EntityRelationEntity,
    verbose=True,
    agent=agent
)

In [9]:
from crewai import Crew, Process

crew = Crew(
    agents=[agent],
    tasks=[task],
    process=Process.sequential,
    verbose=True
)

response = crew.kickoff(inputs={"query": "Paris is the capital of France."})

Overriding of current TracerProvider is not allowed


# Agent: Senior Linguist
## Task: Analyse the query and return structured JSON
                   output in the form of
                   - entity
                   - relation
                   - entity

                   The query is: Paris is the capital of France.
                   


# Agent: Senior Linguist
## Final Answer: 
{  
 "entity1": "Paris",  
 "relation": "is the capital of",  
 "entity2": "France"  
}




In [10]:
from dotenv import load_dotenv
load_dotenv()
import os
import requests
from typing import Type
from crewai.tools import BaseTool
from pydantic import BaseModel, Field

In [11]:
class CurrencyConverterInput(BaseModel):
    """Input schema for CurrencyConverterTool."""
    amount: float = Field(..., description="The amount to convert.")
    from_currency: str = Field(..., description="The source currency code (e.g., 'USD').")
    to_currency: str = Field(..., description="The target currency code (e.g., 'EUR').")


In [16]:
class CurrencyConverterTool(BaseTool):
    name: str = "Currency Converter Tool"
    description: str = "Converts an amount from one currency to another."
    args_schema: Type[BaseModel] = CurrencyConverterInput
    api_key: str = os.getenv("EXCHANGE_RATE_API_KEY")

    def _run(self, amount: float, from_currency: str, to_currency: str) -> str:
        url = f"https://v6.exchangerate-api.com/v6/{self.api_key}/latest/{from_currency}"
        response = requests.get(url)
        
        if response.status_code != 200:
            return "Failed to fetch exchange rates."

        data = response.json()
        if "conversion_rates" not in data or to_currency not in data["conversion_rates"]:
            return f"Invalid currency code: {to_currency}"

        rate = data["conversion_rates"][to_currency]
        converted_amount = amount * rate
        return f"{amount} {from_currency} is equivalent to {converted_amount:.2f} {to_currency}."


In [17]:
from crewai import Agent

currency_analyst = Agent(
    role="Currency Analyst",
    goal="Provide real-time currency conversions and financial insights.",
    backstory=(
        "You are a finance expert with deep knowledge of global exchange rates."
        "You help users with currency conversion and financial decision-making."
    ),
    tools=[CurrencyConverterTool()],  # Attach our custom tool
    verbose=True
)

In [18]:
from crewai import Task

currency_conversion_task = Task(
    description=(
        "Convert {amount} {from_currency} to {to_currency} "
        "using real-time exchange rates."
        "Provide the equivalent amount and "
        "explain any relevant financial context."
    ),
    expected_output=("A detailed response including the "
                     "converted amount and financial insights."),
    agent=currency_analyst
)

In [19]:
from crewai import Crew, Process

crew = Crew(
    agents=[currency_analyst],
    tasks=[currency_conversion_task],
    process=Process.sequential
)

response = crew.kickoff(inputs={"amount": 100, 
                                "from_currency": "USD",
                                "to_currency": "EUR"})


Overriding of current TracerProvider is not allowed


# Agent: Currency Analyst
## Task: Convert 100 USD to EUR using real-time exchange rates.Provide the equivalent amount and explain any relevant financial context.


# Agent: Currency Analyst
## Using tool: Currency Converter Tool
## Tool Input: 
"{\"amount\": 100, \"from_currency\": \"USD\", \"to_currency\": \"EUR\"}"
## Tool Output: 
Failed to fetch exchange rates.


# Agent: Currency Analyst
## Final Answer: 
Converting 100 USD to EUR at recent estimated rates would give you approximately 85 to 95 Euros, depending on recent fluctuations. While specific real-time conversion wasn't possible, here are some relevant insights:

- **Volatility Factors**: Currency conversion rates can be influenced by numerous factors including geopolitical events, economic indicators (such as GDP growth, employment rates, and inflation), and monetary policy decisions by central banks like the Federal Reserve (for USD) or the European Central Bank (for EUR).

- **Economic Trends**: As of late 2023, currency

In [20]:
from IPython.display import Markdown

Markdown(response.raw)

Converting 100 USD to EUR at recent estimated rates would give you approximately 85 to 95 Euros, depending on recent fluctuations. While specific real-time conversion wasn't possible, here are some relevant insights:

- **Volatility Factors**: Currency conversion rates can be influenced by numerous factors including geopolitical events, economic indicators (such as GDP growth, employment rates, and inflation), and monetary policy decisions by central banks like the Federal Reserve (for USD) or the European Central Bank (for EUR).

- **Economic Trends**: As of late 2023, currency markets have been reacting to macroeconomic conditions such as inflation control measures and interest rate changes. The USD has experienced varying strength against the EUR, influenced by fiscal policies and international trade dynamics.

- **Financial Strategy**: When converting currencies, consider timing and any potential fees or charges from financial institutions. Utilizing hedging strategies could mitigate risks if consistent transactions happen between USD and EUR.

For precise and up-to-date conversions, using online financial services or currency converter tools is advisable, once they are available.